# ui

> Fill in a module description here

In [ ]:
#| default_exp ui

In [ ]:
#| hide
#from nbdev.showdoc import *

In [1]:
#| export

from termcolor import colored
from random import shuffle, randint, sample
import emoji
import tabulate

In [4]:
#| export
import clue.model as model

In [6]:
#| export
def cardFull(info):
    name,color, icon = info[:3]
    if len(icon) > 0:
        ret = icon + " " + colored(name,color)
    else:
        ret = colored(name,color)
    
    return ret

def cardShort(info):
    if len(info) < 3:
        return cardFull(info)
    color,icon, name = info[1:]
    if len(icon) > 0:
        ret = icon + " " + colored(name,color)
    else:
        ret = colored(name,color)
   
    return  ret


In [7]:
#| hide
sampleC = ["hi","red",u"\U0001f460","Scar."]

cardFull(sampleC),cardShort(sampleC)

('👠 hi', '👠 Scar.')

In [8]:
#| export
def cIndentity(x):
  return x[0]
    
def printHand(hand,view=cIndentity):
    container = []
    line = [view(card) for card in hand]
    print(', '.join(line))

def printPlayerHand(player,game):
    hand = [model.keyLookUp(x) for x in model.handOf(player,game)]
    printHand(hand,cardFull)
            


In [9]:
#| hide
#printHand(model.people)
pgame = model.createClue(6)
printPlayerHand(2,pgame)

Library, 🔧 Wrench, Kitchen


In [10]:
#| hide
printHand(model.people,cardFull)
printHand(model.people,cardShort)

👠 Miss Scarlet, 🧐 Colonel Mustard, ♡  Mrs. White, 🐉 Mr. Green, 🍆 Professor Plum, 🦚 Mrs. Peacock
👠 Scar., 🧐 Must., ♡ White, 🐉 Green, 🍆 Plum , 🦚 Peac.


In [11]:
#| export



def printPlayer(game,i):
    ploc = game[-1]
    person = model.people[i-1]
    d = cardFull(person)
    loc = cardFull(model.cardLookup(3,ploc[i-1]))
    d = "[" + str(i) + "] " + d  + " is in " + loc 
    print("\n" + d.center(70," "))
 
    ahand = game[i]
    maps = [model.keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)

    
def printGame(game):
    print("Solution")
    ahand = game[0]
   
    maps = [model.keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)
    for i in range(1,len(game)-1):
        printPlayer(game,i)
        


     
       

In [12]:
#| hide
agame = model.createClue(4)
amoves = []
printGame(agame)
agame[-1], model.rooms

Solution
🧐 Colonel Mustard, 🔧 Wrench, Lounge

                 [1] 👠 Miss Scarlet is in Dining Room                 
Library, 👠 Miss Scarlet, Conservatory, 🍆 Professor Plum, Dining Room

                  [2] 🧐 Colonel Mustard is in Lounge                  
Billiard Room, 🔫 Revolver, Kitchen, Study, 🕯️ Candlestick

                   [3] ♡  Mrs. White is in Library                    
Ball Room, 🔪 Knife, 🪢 Rope, ♡  Mrs. White

                      [4] 🐉 Mr. Green is in Hall                      
🪠 Lead Pipe, Hall, 🦚 Mrs. Peacock, 🐉 Mr. Green


([3, 6, 7, 4, 0, 2],
 [['Ball Room', 'cyan', '', 'Ball'],
  ['Billiard Room', 'cyan', '', 'Bill'],
  ['Conservatory', 'cyan', '', 'Consv'],
  ['Dining Room', 'cyan', '', 'Dining'],
  ['Hall', 'cyan', '', 'Hall'],
  ['Kitchen', 'cyan', '', 'Kitch'],
  ['Lounge', 'cyan', '', 'Lounge'],
  ['Library', 'grey', '', 'Lib'],
  ['Study', 'grey', '', 'Study']])

In [ ]:
#| hide


## Response and Answer
    
        


In [13]:
#| export
def showChoice(guess):
    p,w,l = guess[:3]
    ret = cardFull(model.cardLookup(1,p))
    ret = ret + " with the " + cardFull(model.cardLookup(2,w))
    ret = ret + " in the " + cardFull(model.cardLookup(3,l))
    return ret
    

In [14]:
#| hide
aguess = (0,2,7)
print(showChoice(aguess))

👠 Miss Scarlet with the 🪢 Rope in the Library


In [15]:
#| hide
aresponse =  model.responseToGuess(aguess,1,4, model.keysForGuess(aguess)[0])
aresponse, showChoice(aresponse)

((0, 2, 7, 1, 4, 100), '👠 Miss Scarlet with the 🪢 Rope in the Library')

In [13]:
#| hide
amoves.append(aresponse)

In [29]:
#| hide
amoves=[]
for x in range(7):
    pc = randint(1,4)
    p = randint(0,5)
    w =  randint(0,5)
    l = randint(0,5)
    aguess = (p,w,l)
    aresponse =  model.getAnswer(pc,agame,aguess)
    if aresponse[0] > -1:
        amoves.append(aresponse)


        

amoves

[(3, 3, 1, 3, 4, 103),
 (0, 0, 0, 4, 1, 100),
 (3, 3, 5, 3, 4, 103),
 (3, 3, 0, 4, 3, 300),
 (4, 1, 1, 2, 1, 104),
 (0, 2, 0, 1, 3, 300),
 (0, 1, 2, 2, 1, 100)]

In [1]:
#| export
def wrapL(s,f,l):
    show = set([s-1])
    s += 1
    if s > l:
        s = 1
    while s != f:
        show.add(s-1)
        s += 1
        if s > l:
            s = 1
    show.add(f-1)
    
    ret = ""
    for x in range(l):
        if x in show:
            ret += model.people[x][2]
        else:
            ret += "❓" 
    return ret
        
def showMoves(pc,moves,plMax):
    for answer in moves:
        p,w,l,asker, answered,shown = answer
        line = model.people[asker-1][2] + "  asks "
        line += "(" + model.people[p][2] + " &  " + model.weapons[w][2] + " in the "       
        line += ( model.rooms[w][0] +") ").ljust(15,"-")
        line += "->" + wrapL(asker,answered,plMax)
        if asker == pc or pc < 0 :
             
            line += " shows " + cardFull(model.cardLookup(int(shown/100),shown%100))
        print(line)
        
#showMoves(1,amoves,4)     
        


In [40]:
emoji.emojize(":question:"),u"\U0001F97C"

(':question:', '🥼')

In [15]:
#| hide
model.getSeen(1,agame,amoves)

[103, 306, 300, 102, 100, 100]

In [ ]:
aresponse

(0, 2, 7, 1, 4, 100)

In [18]:
#| hide
g = model.passedOnResponse(aresponse,agame)

In [1]:
#| export
def iconOfPlayers(pList):
    pList.sort()
    return ", ".join([model.people[x-1][2] for x in pList if x>0] )
    

In [19]:
#| hide
iconOfPlayers(g)

'🧐, ♡'

In [20]:
b = model.notInHands(agame,amoves)


In [1]:
print(amoves)

NameError: name 'amoves' is not defined

## Input

In [22]:
def chooseCard(hand,view=cIndentity,fmt="fancy_grid",flag = {}):
    container = []
    i = 1
    ml = max([len(x[0]) for x in hand])

    no = " "
    
    for card in hand:
        face = view(card)
        #face.append(i)
        color = card[1]
        no = " "
        if card[0] in flag:
            no = "*"
        container.append(colored(str(i) +")" + no + " " ,color) + face)
        i += 1
    choice = -1
    
    while choice < 1:
        for card in container:
            print(card)
        choice = int(input("Choose from 1 to " +  str(len(container))+": "))
       
        if (choice > 0) and (choice <= len(container)):
            return choice
        print(str(choice) + " is not in range")
        choice = -1

In [23]:
#| hide
chooseCard(model.people,cardFull,flag = {"Mr. Green"})

1)  👠 Miss Scarlet
2)  🧐 Colonel Mustard
3)  ♡  Mrs. White
4)* 🐉 Mr. Green
5)  🍆 Professor Plum
6)  🦚 Mrs. Peacock


Choose from 1 to 6:  2


2

In [24]:
#| hide
chooseCard(model.people,cardShort,flag = {"Mr. Green"})

1)  👠 Scar.
2)  🧐 Must.
3)  ♡ White
4)* 🐉 Green
5)  🍆 Plum 
6)  🦚 Peac.


Choose from 1 to 6:  2


2

In [25]:
#| export
def chooseKey(keys,playerID,game,moves):
    container = []
    i = 1
    #ml = max([len(x[0]) for x in hand])

    passedOn = model.notInHands(game,moves)
    flagged = set(model.getSeen(playerID, game, moves))
    keyChoice = -1

    while keyChoice < 1:   
        i = 1
        for pair in keys:
            no = " "
            key = model.keyForArray(pair)
        
            if key in flagged:
                no = "*"
            data = model.cardForArray(pair)
            view = cardFull(data)
        
            base = str(i) + no + view
            if key in passedOn:
                shownTo = passedOn[key]
                base = base.ljust(35)  + iconOfPlayers(list(shownTo))
                
            print(base)
            i+=1
        
    

        keyChoice = int(input("Choose from 1 to " +  str(len(keys))+": "))
       
        if (keyChoice > 0) and (keyChoice <= len(keys)):
            return keyChoice
        print(str(keyChoice) + " is not in range")
        keyChoice = -1
    return keyChoice
    


In [26]:
wKeys = [[1,x] for x in range(len(model.people))]
chooseKey(wKeys,1,agame,amoves)

1*👠 Miss Scarlet                   🧐, ♡
2 🧐 Colonel Mustard
3*♡  Mrs. White
4*🐉 Mr. Green
5 🍆 Professor Plum
6 🦚 Mrs. Peacock


Choose from 1 to 6:  2


2

In [27]:
wKeys = [[1,x] for x in range(len(model.people))]
chooseKey(wKeys,2,agame,amoves)

1 👠 Miss Scarlet                   🧐, ♡
2 🧐 Colonel Mustard
3 ♡  Mrs. White
4 🐉 Mr. Green
5 🍆 Professor Plum
6*🦚 Mrs. Peacock


Choose from 1 to 6:  3


3

## board mechanics

In [ ]:
#| export
def playerMarker(i):
    info = model.people[i]
    color,icon, name = info[1:]

    return colored(name[0],color)
   
    


In [ ]:
print(playerMarker(0))

S


In [ ]:
def notInHands(game,moves):
    ret = dict()
    
    for x in moves:
        
        
        thePasses = passedOnResponse(x,game) 
        
        if len(thePasses) > 0:
            shown = x[-1]
            if shown in ret:
                addToMe = ret[shown]
            else:
                addToMe = set()
            for aPass in thePasses:
                addToMe.add(aPass)
            ret[shown] = addToMe
                
       
    return ret

In [ ]:
#| export
def roomFor(i,ch,where,visited):
    #print(i)
 
    if i<1:
        return ""
    adds = ""
    
    peeps = ", ".join([playerMarker(x) for x in where[i-1]])

    try:
        x = ch.index(i-1)
        adds = str(x+1) +") "
    except:
        adds = ""
        #print(i)

    roomName = model.rooms[i-1][0]
    k = model.keyCard(3,i-1)
    
    if k in visited:
        roomName = colored(roomName,"red")
        

    
    return adds + roomName +"\n   " + peeps
    
def showBoard(pc,game,moves):
    roomCount = len(model.rooms)
    
    where = [[] for x in range(roomCount)]
    ploc = game[-1]
    #print(ploc)
    for i in range(len(ploc)):
        suspect = i + 1
        room = model.roomOf(suspect,game)
        where[room].append(i)

    visited  = set(model.getSeen(pc,game, moves))
    
    lines = [[1,2,3],[9,0,4],[8,0,5],[0,7,6]]
    
    playerRoom = model.roomOf(pc,game)
    b = model.roomsAround(playerRoom)

    m = []

    for x in lines:
        m.append([roomFor(i,b,where,visited) for i in x])
    print(tabulate.tabulate(m,  tablefmt="fancy_grid"))

In [ ]:
showBoard(1,agame,amoves)

╒══════════════╤═══════════════╤══════════════╕
│ 3) Ball Room │ Billiard Room │ Conservatory │
│              │    M          │    W         │
├──────────────┼───────────────┼──────────────┤
│ 2) Study     │               │ Dining Room  │
│    S         │               │    P         │
├──────────────┼───────────────┼──────────────┤
│ 1) Library   │               │ Hall         │
│              │               │    G         │
├──────────────┼───────────────┼──────────────┤
│              │ Lounge        │ Kitchen      │
│              │               │    P         │
╘══════════════╧═══════════════╧══════════════╛


In [ ]:
#| export
def showLocations(game):
    ploc = game[-1]

    for i in range(len(ploc)):
        pc = i + 1
        person = model.people[i]
        d = cardFull(person)
        room = model.roomOf(pc,game)
        loc = cardFull(model.cardLookup(3,room))
        d = "[" + str(i+1) + "] " + d  + " is in " + loc 
        print(d)

In [ ]:
#| hide
showLocations(agame)
printGame(agame)

[1] 👠 Miss Scarlet is in Study
[2] 🧐 Colonel Mustard is in Billiard Room
[3] ♡  Mrs. White is in Conservatory
[4] 🐉 Mr. Green is in Hall
[5] 🍆 Professor Plum is in Kitchen
[6] 🦚 Mrs. Peacock is in Dining Room
Solution
🦚 Mrs. Peacock, 🪢 Rope, Billiard Room

           [1] 👠 Miss Scarlet is in Study           
Library, 🍆 Professor Plum, 🔪 Knife, Dining Room, 🔧 Wrench

     [2] 🧐 Colonel Mustard is in Billiard Room      
Ball Room, Conservatory, Lounge, 🪠 Lead Pipe, Hall

        [3] ♡  Mrs. White is in Conservatory        
👠 Miss Scarlet, Kitchen, 🐉 Mr. Green, 🕯️ Candlestick

             [4] 🐉 Mr. Green is in Hall             
🔫 Revolver, ♡  Mrs. White, Study, 🧐 Colonel Mustard


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()